In [125]:
import numpy as np
import pandas as pd
import re
import urllib.request
import time
import tensorflow_datasets as tfds
import tensorflow as tf

In [ ]:
urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData%20.csv", filename="ChatBotData.csv")
train_data = pd.read_csv('ChatBotData.csv')

In [ ]:
questions = []
for sentence in train_data['Q']:
  sentence = re.sub(r"([?.!,])", r"\1", sentence)
  sentence = sentence.strip()
  questions.append(sentence)

In [ ]:
answers = []
for sentence in train_data['A']:
  sentence = re.sub(r"([?.!,])", r"\1", sentence)
  sentence = sentence.strip()
  answers.append(sentence)

In [ ]:
tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(questions+answers, target_vocab_size=2**13)

In [ ]:
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size+1]
VOCAB_SIZE = tokenizer.vocab_size + 2

In [ ]:
MAX_LENGTH = 40

def tokenize_and_filter(inputs, outputs):
  tokenized_inputs, tokenized_outputs = [], []

  for (sentence1, sentence2) in zip(inputs, outputs):
    sentence1 = START_TOKEN + tokenizer.encode(sentence1) + END_TOKEN
    sentence2 = START_TOKEN + tokenizer.encode(sentence2) + END_TOKEN
    
    tokenized_inputs.append(sentence1)
    tokenized_outputs.append(sentence2)

  tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(tokenized_inputs, maxlen=MAX_LENGTH, padding='post')
  tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(tokenized_outputs, maxlen=MAX_LENGTH, padding='post')
  return tokenized_inputs, tokenized_outputs

questions, answers = tokenize_and_filter(questions, answers)

In [ ]:
BATCH_SIZE = 64
BUFFER_SIZE = 20000

dataset = tf.data.Dataset.from_tensor_slices(({'inputs':questions,'dec_inputs':answers[:,0:-1]},{'outputs':answers[:,1:]},))
dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
class PositionalEncoding(tf.keras.layers.Layer):
  def __init__(self, position, d_model):
    super(PositionalEncoding, self).__init__()
    self.pos_encoding = self.positional_encoding(position, d_model)

  def get_angles(self, position, i, d_model):
    angles = 1 / tf.pow(10000, (2*(i//2))/tf.cast(d_model, tf.float32))
    return position * angles
  
  def positional_encoding(self, position, d_model):
    angle_rads = self.get_angles(position = tf.range(position, dtype=tf.float32)[:,tf.newaxis],i = tf.range(d_model, dtype=tf.float32)[tf.newaxis,:],d_model = d_model)
    
    sines = tf.math.sin(angle_rads[:,0::2])
    cosines = tf.math.cos(angle_rads[:,1::2])

    angle_rads = np.zeros(angle_rads.shape)
    angle_rads[:,0::2] = sines
    angle_rads[:,1::2] = cosines
    pos_encoding = tf.constant(angle_rads)
    pos_encoding = pos_encoding[tf.newaxis, ...]

    print(pos_encoding.shape)
    return tf.cast(pos_encoding, tf.float32)

  def call(self, inputs):
    return inputs + self.pos_encoding[:,:tf.shape(inputs)[1],:]

In [ ]:
def scaled_dot_product_attention(query, key, value, mask):
  attention_score = tf.matmul(query, key, transpose_b=True) / tf.cast(tf.shape(key)[-1], tf.float32)
  
  if mask is not None:
    attention_score += (mask * 1e-9)

  attention_weights = tf.nn.softmax(attention_score, axis=-1)
  output = tf.matmul(attention_weights, value)
  return output, attention_weights

In [ ]:
class MultiHeadAttention(tf.keras.layers.Layer):
  def __init__(self, d_model, num_heads, name="multi_head_attention"):
    super(MultiHeadAttention, self).__init__(name=name)
    self.d_model = d_model
    self.num_heads = num_heads

    assert d_model % self.num_heads == 0

    self.depth = d_model // self.num_heads

    self.query_dense = tf.keras.layers.Dense(units=d_model)
    self.key_dense = tf.keras.layers.Dense(units=d_model)
    self.value_dense = tf.keras.layers.Dense(units=d_model)

    self.dense = tf.keras.layers.Dense(units=d_model)

  def split_heads(self, inputs, batch_size):
    inputs = tf.reshape(inputs, shape=(batch_size, -1, self.num_heads, self.depth))
    return tf.transpose(inputs, perm=[0,2,1,3])
  
  def call(self, inputs):
    query, key, value, mask = inputs['query'], inputs['key'], inputs['value'], inputs['mask']
    batch_size = tf.shape(query)[0]
    
    query = self.query_dense(query)
    key = self.key_dense(key)
    value = self.value_dense(value)

    query = self.split_heads(query, batch_size)
    key = self.split_heads(key, batch_size)
    value = self.split_heads(value, batch_size)

    scaled_attention, _ = scaled_dot_product_attention(query, key, value, mask)
    scaled_attention = tf.transpose(scaled_attention, perm=[0,2,1,3])

    concat_attention = tf.reshape(scaled_attention, (batch_size,-1,self.d_model))

    outputs = self.dense(concat_attention)
    return outputs

In [ ]:
def create_padding_mask(x):
  mask = tf.cast(tf.math.equal(x,0), tf.float32)
  return mask[:, tf.newaxis, tf.newaxis, :]

In [ ]:
def encoder_layer(dff, d_model, num_heads, dropout, name="encoder_layer"):
  inputs = tf.keras.Input(shape=(None,d_model), name="inputs")

  padding_mask = tf.keras.Input(shape=(1,1,None), name="padding_mask")

  attention = MultiHeadAttention(d_model, num_heads, name="attention")({'query':inputs, 'key':inputs, 'value':inputs, 'mask':padding_mask})

  attention = tf.keras.layers.Dropout(rate=dropout)(attention)
  attention = tf.keras.layers.LayerNormalization(epsilon=1e-6)(inputs+attention)

  outputs = tf.keras.layers.Dense(units=dff, activation='relu')(attention)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)

  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(epsilon=1e-6)(attention+outputs)

  return tf.keras.Model(inputs=[inputs,padding_mask], outputs=outputs, name=name)

In [ ]:
def encoder(vocab_size, num_layers, dff, d_model, num_heads, dropout, name="encoder"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")

  padding_mask = tf.keras.Input(shape=(1,1,None), name="padding_mask")

  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)
  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  for i in range(num_layers):
    outputs = encoder_layer(dff=dff, d_model=d_model, num_heads=num_heads, dropout=dropout, name="encoder_layer_{}".format(i),)([outputs, padding_mask])

  return tf.keras.Model(inputs=[inputs,padding_mask], outputs=outputs, name=name)

In [ ]:
def create_look_ahead_mask(x):
  seq_len = tf.shape(x)[1]
  look_ahead_mask = 1 - tf.linalg.band_part(tf.ones((seq_len,seq_len)),-1,0)
  padding_mask = create_padding_mask(x)
  return tf.maximum(look_ahead_mask, padding_mask)

In [ ]:
def decoder_layer(dff, d_model, num_heads, dropout, name="decoder_layer"):
  inputs = tf.keras.Input(shape=(None,d_model), name="inputs")
  enc_outputs = tf.keras.Input(shape=(None,d_model), name="encoder_outputs")

  look_ahead_mask = tf.keras.Input(shape=(1,None,None), name="look_ahead_mask")

  padding_mask = tf.keras.Input(shape=(1,1,None), name="padding_mask")

  attention1 = MultiHeadAttention(d_model, num_heads, name="attention_1")(inputs = {'query':inputs,'key':inputs,'value':inputs,'mask':look_ahead_mask})
  attention1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(attention1+inputs)

  attention2 = MultiHeadAttention(d_model, num_heads, name="attention_2")(inputs = {'query':attention1,'key':enc_outputs,'value':enc_outputs,'mask':padding_mask})
  attention2 = tf.keras.layers.Dropout(rate=dropout)(attention2)
  attention2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)(attention2+attention1)

  outputs = tf.keras.layers.Dense(units=dff, activation='relu')(attention2)
  outputs = tf.keras.layers.Dense(units=d_model)(outputs)
  outputs = tf.keras.layers.Dropout(rate=dropout)(outputs)
  outputs = tf.keras.layers.LayerNormalization(epsilon=1e-6)(outputs+attention2)

  return tf.keras.Model(inputs=[inputs,enc_outputs,look_ahead_mask,padding_mask], outputs=outputs, name=name)

In [ ]:
def decoder(vocab_size, num_layers, dff, d_model, num_heads, dropout, name='decoder'):
  inputs = tf.keras.Input(shape=(None,), name='inputs')
  enc_outputs = tf.keras.Input(shape=(None,d_model), name='encoder_outputs')
  
  look_ahead_mask = tf.keras.Input(shape=(1,None,None), name='look_ahead_mask')
  
  padding_mask = tf.keras.Input(shape=(1,1,None), name='padding_mask')

  embeddings = tf.keras.layers.Embedding(vocab_size, d_model)(inputs)
  embeddings *= tf.math.sqrt(tf.cast(d_model, tf.float32))
  embeddings = PositionalEncoding(vocab_size, d_model)(embeddings)
  outputs = tf.keras.layers.Dropout(rate=dropout)(embeddings)

  for i in range(num_layers):
    outputs = decoder_layer(dff=dff, d_model=d_model, num_heads=num_heads, dropout=dropout, name='decoder_layer_{}'.format(i),)(inputs=[outputs,enc_outputs,look_ahead_mask,padding_mask])

  return tf.keras.Model(inputs=[inputs,enc_outputs,look_ahead_mask,padding_mask], outputs=outputs, name=name)

In [ ]:
def transformer(vocab_size, num_layers, dff, d_model, num_heads, dropout, name="transformer"):
  inputs = tf.keras.Input(shape=(None,), name="inputs")

  dec_inputs = tf.keras.Input(shape=(None,), name="dec_inputs")

  enc_padding_mask = tf.keras.layers.Lambda(create_padding_mask, output_shape=(1,1,None), name="enc_padding_mask")(inputs)

  look_ahead_mask = tf.keras.layers.Lambda(create_look_ahead_mask, output_shape=(1,None,None), name="look_ahead_mask")(dec_inputs)

  dec_padding_mask = tf.keras.layers.Lambda(create_padding_mask, output_shape=(1,1,None), name="dec_padding_mask")(inputs)

  enc_outputs = encoder(vocab_size=vocab_size, num_layers=num_layers, dff=dff, d_model=d_model, num_heads=num_heads, dropout=dropout,)(inputs=[inputs,enc_padding_mask])

  dec_outputs = decoder(vocab_size=vocab_size, num_layers=num_layers, dff=dff, d_model=d_model, num_heads=num_heads, dropout=dropout,)(inputs=[dec_inputs,enc_outputs,look_ahead_mask,dec_padding_mask])

  outputs = tf.keras.layers.Dense(units=vocab_size, name="outputs")(dec_outputs)
  return tf.keras.Model(inputs=[inputs,dec_inputs], outputs=outputs, name=name)

In [ ]:
NUM_LAYERS = 2
DFF = 512
D_MODEL = 128
NUM_HEADS = 8
DROPOUT = 0.1
my_transformer = transformer(vocab_size=VOCAB_SIZE, num_layers=NUM_LAYERS, dff=DFF, d_model=D_MODEL, num_heads=NUM_HEADS, dropout=DROPOUT, name='my_transformer')

(1, 8172, 128)
(1, 8172, 128)


In [ ]:
def loss_function(y, y_pred):
  y = tf.reshape(y, shape=(-1,MAX_LENGTH-1))

  loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True,reduction='none')(y, y_pred)
  mask = tf.cast(tf.not_equal(y,0), tf.float32)
  loss = tf.multiply(loss, mask)

  return tf.reduce_mean(loss)

In [ ]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super(CustomSchedule, self).__init__()
    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)
    self.warmup_steps = warmup_steps
  
  def __call__(self, step):
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps**-1.5)
    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1,arg2)

In [ ]:
learning_rate = CustomSchedule(D_MODEL)
optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y, y_pred):
  y = tf.reshape(y, shape=(-1,MAX_LENGTH-1))
  return tf.keras.metrics.sparse_categorical_accuracy(y, y_pred)
  
my_transformer.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

In [ ]:
EPOCHS = 1
my_transformer.fit(dataset, epochs=EPOCHS)

185/185 [==============================] - 180s 933ms/step - loss: 1.5449 - accuracy: 0.0103


In [ ]:
def evaluate(sentence):
  sentence = preprocess_sentence(sentence)

  sentence = tf.expand_dims(
      START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)

  output = tf.expand_dims(START_TOKEN, 0)

  for i in range(MAX_LENGTH):
    predictions = my_transformer(inputs=[sentence, output], training=False)

    predictions = predictions[:, -1:, :]
    predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

    if tf.equal(predicted_id, END_TOKEN[0]):
      break

    output = tf.concat([output, predicted_id], axis=-1)

  return tf.squeeze(output, axis=0)

In [ ]:
def predict(sentence):
  prediction = evaluate(sentence)

  predicted_sentence = tokenizer.decode([i for i in prediction if i < tokenizer.vocab_size])

  print('Input: {}'.format(sentence))
  print('Output: {}'.format(predicted_sentence))

  return predicted_sentence

In [ ]:
def preprocess_sentence(sentence):
  sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
  sentence = sentence.strip()
  return sentence

In [ ]:
output=predict("게임하자")

Input: 게임하자
Output: 
